In [1]:
# Importing packages required for the clipping algorith
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import shutil

In [2]:
overlay_path = "C:/Users/Dell/ACASA/public/CountryBoundary/IN.json"

# Check if the file exists
print(f"File exists: {os.path.exists(overlay_path)}")

# Check absolute path
absolute_path = os.path.abspath(overlay_path)
print(f"Absolute path: {absolute_path}")
print(f"File exists at absolute path: {os.path.exists(absolute_path)}")


File exists: True
Absolute path: C:\Users\Dell\ACASA\public\CountryBoundary\IN.json
File exists at absolute path: True


In [3]:
directory = "C:/Users/Dell/ACASA/public/CountryBoundary"

if os.path.exists(directory):
    print("Files in directory:", os.listdir(directory))
else:
    print(f"Directory '{directory}' does not exist.")


Files in directory: ['AF.json', 'BD.json', 'BT.json', 'IN.json', 'MV.json', 'NP.json', 'PK.json', 'SA_Country.json', 'SA_outline.json', 'SL.json']


In [4]:
def clip_maps(input_folder, overlay_path, output_folder, country_code):
    # Read and preprocess the overlay file
    overlay = gpd.read_file(overlay_path).to_crs("EPSG:4326")  # Ensure overlay is in WGS 84
    print(overlay.geometry)

    # Walk through all files and subdirectories
    for root, _, files in os.walk(input_folder):
        for file in files:
            input_path = os.path.join(root, file)
            
            # Preserve folder structure in output
            relative_path = os.path.relpath(root, input_folder)
            output_subfolder = os.path.join(output_folder, relative_path)
            os.makedirs(output_subfolder, exist_ok=True)

            if file.endswith((".shp", ".geojson")):
                # Clip vector files
                try:
                    data = gpd.read_file(input_path)
                    data = data.to_crs(overlay.crs)  # Ensure CRS matches

                    clipped_data = gpd.clip(data, overlay)
                    if clipped_data.empty:
                        print(f"Skipping {file}: Clipping resulted in empty geometry")
                        continue

                    output_path = os.path.join(output_subfolder, f"{country_code}_{file}")
                    clipped_data.to_file(output_path)
                    print(f"Saved clipped vector: {output_path}")

                except Exception as e:
                    print(f"Error clipping vector {file}: {e}")

            elif file.endswith((".tif", ".tiff")):
                # Clip raster files
                try:
                    with rasterio.open(input_path) as src:
                        overlay = overlay.to_crs(src.crs)  # Convert overlay to raster CRS
                        geom = [feature.__geo_interface__ for feature in overlay.geometry]  # Convert to GeoJSON-like format

                        out_image, out_transform = mask(src, geom, crop=True)
                        out_meta = src.meta.copy()

                        # Update metadata
                        out_meta.update({
                            "driver": "GTiff",
                            "height": out_image.shape[1],
                            "width": out_image.shape[2],
                            "transform": out_transform
                        })

                        output_path = os.path.join(output_subfolder, f"{country_code}_{file}")
                        with rasterio.open(output_path, "w", **out_meta) as dest:
                            dest.write(out_image)
                        print(f"Saved clipped raster: {output_path}")

                except Exception as e:
                    print(f"Error clipping raster {file}: {e}")

In [5]:
clip_maps(
    "C:/Users/Dell/ACASA/public/Adap",
    "C:/Users/Dell/ACASA/public/CountryBoundary/IN.json",
    "C:/Users/Dell/ACASA/public/India_Clipped/Adap",
    "IN")

0    MULTIPOLYGON (((93.7048 7.22185, 93.70085 7.21...
Name: geometry, dtype: geometry
Saved clipped raster: C:/Users/Dell/ACASA/public/India_Clipped/Adap\Chickpea\Baseline\IN_Suitability_Chickpea_ADPTI.tif
Saved clipped raster: C:/Users/Dell/ACASA/public/India_Clipped/Adap\Chickpea\Baseline\IN_Suitability_Chickpea_ADVAR.tif
Saved clipped raster: C:/Users/Dell/ACASA/public/India_Clipped/Adap\Chickpea\Baseline\IN_Suitability_Chickpea_BBFIB.tif
Saved clipped raster: C:/Users/Dell/ACASA/public/India_Clipped/Adap\Chickpea\Baseline\IN_Suitability_Chickpea_FRT.tif
Saved clipped raster: C:/Users/Dell/ACASA/public/India_Clipped/Adap\Chickpea\Baseline\IN_Suitability_Chickpea_INSUR.tif
Saved clipped raster: C:/Users/Dell/ACASA/public/India_Clipped/Adap\Chickpea\Baseline\IN_Suitability_Chickpea_LASLV.tif
Saved clipped raster: C:/Users/Dell/ACASA/public/India_Clipped/Adap\Chickpea\Baseline\IN_Suitability_Chickpea_MICIR.tif
Saved clipped raster: C:/Users/Dell/ACASA/public/India_Clipped/Adap\Chickpe